In [1]:
import os
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import HTML
from sklearn.model_selection import train_test_split
from torch import nn, einsum
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from typing import Union, List
import numpy as np
from timm.models.layers import trunc_normal_
import pytorch_lightning as pl 
import monai
#import pytorch_lightning as pl
from monai import transforms
from monai.data import Dataset
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from monai.config import KeysCollection
from monai.losses import DiceLoss, DiceFocalLoss, DiceCELoss
from monai.metrics import DiceMetric, HausdorffDistanceMetric, ConfusionMatrixMetric, MeanIoU
from monai.inferers import sliding_window_inference
from monai.utils import set_determinism
from monai.data import decollate_batch
#from monai.data import NiftiSaver, write_nifti
from monai.transforms import (
    Activations,
    Activationsd,
    AsDiscrete,
    AsDiscreted,
    Compose,
    Invertd,
    LoadImaged,
    MapTransform,
    NormalizeIntensityd,
    Orientationd,
    RandFlipd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    RandSpatialCropd,
    Resized,
    Spacingd,
    ScaleIntensityd,
    EnsureChannelFirstd,
    EnsureTyped,
    EnsureType,
    ConvertToMultiChannelBasedOnBratsClassesd,
    SpatialPadd,
    ScaleIntensityRangePercentilesd,
)

import pandas as pd
#IMPORT THE RES SWIN UNET MODEL 

from ResSwinUnet import swinUnet_t_3D_Residual

from SwinUnet_3D import swinUnet_t_3D

from monai.networks.nets import UNETR, UNet, VNet, DynUNet, SegResNet, SwinUNETR, AttentionUnet

2024-11-05 22:12:02.824570: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-05 22:12:02.870646: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Total number of trainable parameters: 48115676
Total number of trainable parameters: 16495826


In [2]:
from medcam import medcam

In [3]:
import nibabel as nib
#from celluloid import Camera
#import torchio as tio

In [4]:

dataset_path = './scratch/MICCAI_BraTS2020_TrainingData'

sample_path = './scratch/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355'

sample_scan_paths = os.listdir(sample_path)


img_names = ['mask','t2','flair','t1ce','t1']

In [5]:
#SAVING PATHS OF ALL 4 MODALITIES AND THE GROUND TRUTH MASK IN SEPARATE LISTS 

flair_paths = []
t1_paths = []
t1ce_paths = []
t2_paths = []
seg_paths = []

for i in os.listdir(dataset_path):
    
    
    for j in os.listdir(os.path.join(dataset_path, i)):
        if(j.endswith('flair.nii.gz')):
            flair_paths.append(os.path.join(dataset_path, i, j))

        elif(j.endswith('t1.nii.gz')):
            t1_paths.append(os.path.join(dataset_path, i, j))

        elif(j.endswith('t1ce.nii.gz')):
            t1ce_paths.append(os.path.join(dataset_path, i, j))

        elif(j.endswith('t2.nii.gz')):
            t2_paths.append(os.path.join(dataset_path, i, j))

        elif(j.endswith('seg.nii.gz')):
            seg_paths.append(os.path.join(dataset_path, i, j))

In [6]:
#WE WANT THE INPUT TO BE FED AS A FOUR CHANNEL IMAGE WHICH CONTAINS ALL 4 MODALITIES 

train_x = []
train_y = []

for i in range(len(flair_paths)):
    
    flair_path = flair_paths[i]
    t1_path = t1_paths[i]
    t1ce_path = t1ce_paths[i]
    t2_path = t2_paths[i]
    seg_path = seg_paths[i]
    
    files = [flair_path,t1_path,t1ce_path,t2_path]
    
    train_x.append(files)
    train_y.append(seg_path)
    
    
    
    

In [7]:
#CREATING A DICTIONARY TO SAVE ALL IMAGES AND LABELS 

train_dict = []

for x, y in zip(train_x, train_y):
    info = {'image': x, 'label': y}
    train_dict.append(info)

In [8]:
seed = 42 
spacings = [1.0, 1.0, 1.0]

#REGION OF INTEREST SIZE = 128*128*128 
RoiSize = [128, 128, 128]  

#SLIDING WINDOW SIZE = 4*4*4
window_size = [it // 32 for it in RoiSize]  


overlap = 0.125

#SINCE WE ARE USING ALL 4 MODALITIES, THE INPUT CHANNELS WILL BE 4
in_channels = 4

l_percent = 0.5
u_percent = 99.5

train_ratio, val_ratio, test_ratio = [0.8, 0.2, 0.0]

#VARY BATCH SIZE AND THREADS (WORKERS) ACCORDING TO THE CAPACITY OF YOUR MACHINE  

BatchSize = 1
NumWorkers = 4  

#CHANGE MAX EPOCH AND MIN EPOCH ACCORDINGLY

max_epoch = 260
min_epoch = 250

'''
The t_max parameter in the Cosine Annealing Learning Rate (LR) scheduler refers to the maximum number of iterations 
(or epochs) after which the learning rate will complete one cosine cycle.

In Cosine Annealing, the learning rate starts at a high value and decreases following a cosine curve until it reaches 
a minimum value (often very close to 0). Once the t_max number of iterations is reached, 
the scheduler restarts the process, and the learning rate begins increasing again.
'''

LRCycle = 10 #tmax parameter to be used for cosine annealing learning rate scheduler 
'''
ED (peritumoral edema) =  2,ET (enhancing tumor) =  4, 
NET (non-enhancing tumor)  1, 0- NOT TUMOR 
WT = ED + ET + NET
TC = ET+NET
ET
'''
n_classes = 3  #TC, ET, WT

lr = 3e-4  

# ValidSegDir = os.path.join(data_path, 'ValidSeg', model_name)
# PredDataDir = os.path.join(data_path, 'Brats2021Pred')
# PredSegDir = os.path.join(data_path, 'PredSeg', model_name)

In [9]:
'''
WT = ED + ET + NET
TC = ET+NET
ET

Below is the class for computing WT, TC and ET 
'''

class ReverseBratsLabel(transforms.Transform):
    def __call__(self, pred):
        pred = pred.bool()
        tc = pred[0, ::]
        wt = pred[1, ::]
        et = pred[2, ::]
        res = torch.zeros(pred.shape[1:])

        # label_4 = et
        # label_2 = wt - tc
        # label_1 = tc - et

        label_4 = et
        label_2 = torch.logical_and(wt, torch.logical_not(tc))
        label_1 = torch.logical_and(tc, torch.logical_not(et))

        res[label_1] = 1
        res[label_2] = 2
        res[label_4] = 4
        return res

In [10]:
#transforms for training data 

train_process = Compose(
            [
                # load 4 Nifti images and stack them together
                LoadImaged(keys=["image", "label"]),
                
                #ensuring that the channels are the first dimension for image and label 
                EnsureChannelFirstd(keys="image"),
                ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),

                #ensuring that both image and labels are of the same orientation
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(keys=["image", "label"], pixdim=spacings,
                         mode=("bilinear", "nearest"), ),
                 

                RandSpatialCropd(keys=["image", "label"], roi_size=RoiSize, random_size=False),
                
                
                #performing flipping with probability of 0.5 along all 3 axes 
                RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
                RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=1),
                RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=2),

                NormalizeIntensityd(keys="image", nonzero=True,
                                    channel_wise=True),
                RandScaleIntensityd(keys="image", factors=0.1, prob=1.0),
                RandShiftIntensityd(keys="image", offsets=0.1, prob=1.0),

                EnsureTyped(keys=["image", "label"]),
            ]
        )

In [11]:
#transforms for validation data 

val_process = Compose(
            [
                LoadImaged(keys=["image", "label"]),
                EnsureChannelFirstd(keys="image"),
                ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
                Orientationd(keys=["image", "label"], axcodes="RAS"),
                Spacingd(
                    keys=["image", "label"],
                    pixdim=spacings,
                    mode=("bilinear", "nearest"),
                ),
                #RandSpatialCropd(keys=["image", "label"], roi_size=RoiSize, random_size=False),
                #Resized(keys=['image','label'], spatial_size=(128,128,128), mode=['area','nearest']),
                #ScaleIntensityd(keys='image', minv=0.0, maxv=1.0), 
                NormalizeIntensityd(keys="image", nonzero=True,
                                    channel_wise=True),
                EnsureTyped(keys=["image", "label"]),
            ]
        )

In [12]:
#transforms for test data 

test_process = Compose([
            LoadImaged(keys=['image','label']),
            EnsureChannelFirstd(keys='image'),
            ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
            Orientationd(keys=['image','label'], axcodes="RAS"),
            Spacingd(keys=["image","label"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
            #RandSpatialCropd(keys=["image", "label"], roi_size=RoiSize, random_size=False),
            #Resized(keys=['image','label'], spatial_size=(128,128,128), mode=['area','nearest']),
            #ScaleIntensityd(keys='image', minv=0.0, maxv=1.0), 
            #NormalizeIntensityd(keys='image', nonzero=True, channel_wise=True),
            EnsureTyped(keys=["image", "label"]),
        ])

In [13]:
num = len(train_dict)
train_num = int(num * train_ratio) #0.8
val_test_num = int(num * val_ratio) #0.2
if train_num + val_test_num != num: 
    remain = num - train_num - val_test_num
    val_test_num += remain
    
    
train_dict, val_dict = random_split(train_dict, [train_num, val_test_num],
                                    generator=torch.Generator().manual_seed(seed))

In [14]:
train_set = Dataset(train_dict, transform=train_process)
val_set = Dataset(val_dict, transform=val_process)

train_loader = DataLoader(train_set, batch_size=BatchSize, num_workers=NumWorkers, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BatchSize, num_workers=NumWorkers, shuffle=False)

In [15]:
model = swinUnet_t_3D_Residual()

In [16]:
#CREATING THE DATA MODULE WHICH UTILIZES THE DATA LOADERS 

class BratsDataModule(pl.LightningDataModule):
    def __init__(self):
        
        super().__init__()
        
    def train_dataloader(self):
        return train_loader

    def val_dataloader(self):
        return val_loader


    

In [17]:
#CREATING THE MODULE WHICH HANDLES THE TRAINING AND VALIDATION PROCESS

class BratsModel(pl.LightningModule):
    def __init__(self):
        super(BratsModel, self).__init__()
        
        #index of model name 
        #self.index = index
        
        #self.learning_rate = learning_rate 
        self.net = model
        #self.model_name = model_names[self.index]
        
    def forward(self,x):
        
        return self.net(x)
        
        
        

    def configure_optimizers(self):
        
        pass
    
    
    #THIS FUNCTION HANDLES TRAINING FOR EVERY TRAINING BATCH
    def training_step(self, batch, batch_idx):
        pass
    
    
    #THIS FUNCTION HANDLES VALIDATION FOR EVERY VALIDATION BATCH
    def validation_step(self, batch, batch_idx):
        pass 
        
    def predict_step(self, batch, batch_idx, dataloader_idx=None):
        pass 
        
            
            
    #THIS FUNCTION LOGS ALL THE METRICS FOR TRAINING AT THE END OF EVERY EPOCH 
    def on_training_epoch_end(self):
        pass 
            
            
    #THIS FUNCTION LOGS ALL THE METRICS FOR VALIDATION AT THE END OF EVERY EPOCH
    def on_validation_epoch_end(self):
        pass 
            
    #THIS FUNCTION HANDLES CALCULATION OF METRICS AT THE END OF EVERY EPOCH

    def shared_epoch_end(self, loss_key, stage: int = 0):  #stage - 0 for training and 1 for validation
        pass 
    #THIS FUNCTION HANDLES CALCULATION OF METRICS AFTER READING EVERY SAMPLE DURING TRAINING AND VALIDATION

    def shared_step(self, y_hat, y, stage: int = 0):  #stage - 0 for training and 1 for validation
        pass 
    
    

In [18]:
def setseed():
    pl.seed_everything(seed)
    set_determinism(seed)

In [19]:
#loading the weights of the pre-trained model 
#change the path accordingly 

save_path = './scratch/Validation_Results_3rd_Trial/ResSWINUnet/BraTS_2020/logs/learning_rate_0.0003/epoch=249-valid_loss=0.1616-wt_valid_mean_dice=0.9170-tc_valid_mean_dice=0.8539-et_valid_mean_dice=0.8031.ckpt'

saved_model = BratsModel.load_from_checkpoint(save_path)

In [21]:
saved_model.net.out

Sequential(
  (0): Conv3d(32, 3, kernel_size=(1, 1, 1), stride=(1, 1, 1))
)

In [22]:
import torch
from monai.transforms import Compose, Resize
from torch.utils.data import DataLoader

# Define the resize transform
resize_transform = Compose([
    Resize(spatial_size=(128, 128, 128), mode='trilinear', align_corners=True)
])

resized_val_loader = []

for batch_idx, batch in enumerate(val_loader):
    x = batch["image"]
    y = batch["label"]
    
    # Initialize a tensor to hold the resized channels
    resized_x = torch.zeros((1, 4, 128, 128, 128), device=x.device)
    
    # Resize each channel separately
    for channel in range(4):
        x_channel = x[0, channel, :, :, :]
        resized_x[0, channel, :, :, :] = resize_transform(x_channel.unsqueeze(0)).squeeze(0)
    
    # Create a new batch with the resized input
    resized_batch = {"image": resized_x, "label": y}
    
    resized_val_loader.append(resized_batch)

# Convert the list of resized batches to a DataLoader
resized_val_loader = DataLoader(resized_val_loader, batch_size=1, shuffle=False)

In [25]:
from monai.visualize import GradCAMpp
import matplotlib.pyplot as plt

#Getting the CAMs for enc12, res12, enc3, res3, enc5, res5 layers 

gcam_enc12 = GradCAMpp(nn_module=saved_model.net, target_layers="enc12")
gcam_res12 = GradCAMpp(nn_module=saved_model.net, target_layers="res12")

gcam_enc3 = GradCAMpp(nn_module=saved_model.net, target_layers="enc3")
gcam_res3 = GradCAMpp(nn_module=saved_model.net, target_layers="res3")

gcam_enc5 = GradCAMpp(nn_module=saved_model.net, target_layers="enc5")
gcam_res5 = GradCAMpp(nn_module=saved_model.net, target_layers="res5")



In [1]:
from monai.visualize import GradCAM
import matplotlib.pyplot as plt

#Getting the CAMs for enc12, res12, enc3, res3, enc5, res5 layers 

gcam_enc12 = GradCAM(nn_module=saved_model.net, target_layers="enc12")
gcam_res12 = GradCAM(nn_module=saved_model.net, target_layers="res12")

gcam_enc3 = GradCAM(nn_module=saved_model.net, target_layers="enc3")
gcam_res3 = GradCAM(nn_module=saved_model.net, target_layers="res3")

gcam_enc5 = GradCAM(nn_module=saved_model.net, target_layers="enc5")
gcam_res5 = GradCAM(nn_module=saved_model.net, target_layers="res5")

for batch_idx,batch in enumerate(resized_val_loader):
    images = batch["image"].to('cuda')
    images = torch.squeeze(images,dim=0)
    
    
    
    # Generate GradCAM for classes 0,1,2  
    cam0_enc12 = gcam_enc12(x=images, class_idx=0)
    cam1_enc12 = gcam_enc12(x=images, class_idx=1)
    cam2_enc12 = gcam_enc12(x=images, class_idx=2)
    
    cam0_res12 = gcam_res12(x=images, class_idx=0)
    cam1_res12 = gcam_res12(x=images, class_idx=1)
    cam2_res12 = gcam_res12(x=images, class_idx=2)
    
    cam0_enc3 = gcam_enc3(x=images, class_idx=0)
    cam1_enc3 = gcam_enc3(x=images, class_idx=1)
    cam2_enc3 = gcam_enc3(x=images, class_idx=2)
    
    cam0_res3 = gcam_res3(x=images, class_idx=0)
    cam1_res3 = gcam_res3(x=images, class_idx=1)
    cam2_res3 = gcam_res3(x=images, class_idx=2)
    
    cam0_enc5 = gcam_enc5(x=images, class_idx=0)
    cam1_enc5 = gcam_enc5(x=images, class_idx=1)
    cam2_enc5 = gcam_enc5(x=images, class_idx=2)
    
    cam0_res5 = gcam_res5(x=images, class_idx=0)
    cam1_res5 = gcam_res5(x=images, class_idx=1)
    cam2_res5 = gcam_res5(x=images, class_idx=2)
    
    cam_images = [cam0_enc12,cam1_enc12,cam2_enc12, 
              cam0_res12,cam1_res12,cam2_res12,
              cam0_enc3,cam1_enc3,cam2_enc3,
              cam0_res3,cam1_res3,cam2_res3,
              cam0_enc5,cam1_enc5,cam2_enc5,
              cam0_res5,cam1_res5,cam2_res5]
    
    
    cam_image_names = ["CAM0 ENC12", "CAM1 ENC12", "CAM2 ENC12",
                       "CAM0 RES12", "CAM1 RES12", "CAM2 RES12",
                       "CAM0 ENC3", "CAM1 ENC3", "CAM2 ENC3",
                       "CAM0 RES3", "CAM1 RES3", "CAM2 RES3",
                       "CAM0 ENC5", "CAM1 ENC5", "CAM2 ENC5",
                       "CAM0 RES5", "CAM1 RES5", "CAM2 RES5"]
    
    
    #print(cam.shape)
    
    # Visualize the GradCAM results
    # We'll visualize the middle slice of each channel
    fig, axes = plt.subplots(2, 7, figsize=(20, 10))
    image_names = ["flair","t1","t1ce","t2"]
    
    
    for j in range(0,2):
        for channel in range(4):
        # Visualizing the Original images
            axes[j,channel].imshow(images[0, channel, :, :, 57].cpu().numpy(), cmap='gray')
            axes[j,channel].set_title(image_names[channel])
            axes[j,channel].axis('off')
        
        #visualizing the CAMs using the jet heatmaps 
        
        for i in range(4,7):
            axes[j,i].imshow(cam_images[-4+i+3*j][0, 0, :, :, 57].cpu().numpy(),cmap='jet', alpha=0.7)
            axes[j,i].set_title(cam_image_names[-4+i+3*j])
            axes[j,i].axis('off')

    
    plt.tight_layout()
    plt.show()

In [ ]:
# from monai.visualize import GradCAM
# import matplotlib.pyplot as plt

# # Assuming 'saved_model' is your loaded model

# gcam_enc12 = GradCAM(nn_module=saved_model.net, target_layers="enc12")
# gcam_res12 = GradCAM(nn_module=saved_model.net, target_layers="res12")

# gcam_enc3 = GradCAM(nn_module=saved_model.net, target_layers="enc3")
# gcam_res3 = GradCAM(nn_module=saved_model.net, target_layers="res3")

# gcam_enc5 = GradCAM(nn_module=saved_model.net, target_layers="enc5")
# gcam_res5 = GradCAM(nn_module=saved_model.net, target_layers="res5")

# for batch_idx,batch in enumerate(resized_val_loader):
#     images = batch["image"].to('cuda')
#     images = torch.squeeze(images,dim=0)
    
    
    
#     # Generate GradCAM 0,1,2 are classes 
#     cam0_enc12 = gcam_enc12(x=images, class_idx=0)
#     cam1_enc12 = gcam_enc12(x=images, class_idx=1)
#     cam2_enc12 = gcam_enc12(x=images, class_idx=2)
    
#     cam0_res12 = gcam_res12(x=images, class_idx=0)
#     cam1_res12 = gcam_res12(x=images, class_idx=1)
#     cam2_res12 = gcam_res12(x=images, class_idx=2)
    
#     cam0_enc3 = gcam_enc3(x=images, class_idx=0)
#     cam1_enc3 = gcam_enc3(x=images, class_idx=1)
#     cam2_enc3 = gcam_enc3(x=images, class_idx=2)
    
#     cam0_res3 = gcam_res3(x=images, class_idx=0)
#     cam1_res3 = gcam_res3(x=images, class_idx=1)
#     cam2_res3 = gcam_res3(x=images, class_idx=2)
    
#     cam0_enc5 = gcam_enc5(x=images, class_idx=0)
#     cam1_enc5 = gcam_enc5(x=images, class_idx=1)
#     cam2_enc5 = gcam_enc5(x=images, class_idx=2)
    
#     cam0_res5 = gcam_res5(x=images, class_idx=0)
#     cam1_res5 = gcam_res5(x=images, class_idx=1)
#     cam2_res5 = gcam_res5(x=images, class_idx=2)
    
#     cam_images = [cam0_enc12,cam1_enc12,cam2_enc12, 
#               cam0_res12,cam1_res12,cam2_res12,
#               cam0_enc3,cam1_enc3,cam2_enc3,
#               cam0_res3,cam1_res3,cam2_res3,
#               cam0_enc5,cam1_enc5,cam2_enc5,
#               cam0_res5,cam1_res5,cam2_res5]
    
    
#     cam_image_names = ["CAM0 ENC12", "CAM1 ENC12", "CAM2 ENC12",
#                        "CAM0 RES12", "CAM1 RES12", "CAM2 RES12",
#                        "CAM0 ENC3", "CAM1 ENC3", "CAM2 ENC3",
#                        "CAM0 RES3", "CAM1 RES3", "CAM2 RES3",
#                        "CAM0 ENC5", "CAM1 ENC5", "CAM2 ENC5",
#                        "CAM0 RES5", "CAM1 RES5", "CAM2 RES5", 
#                        "NULL", "NULL"]
    
    
#     #print(cam.shape)
    
#     # Visualize the GradCAM results
#     # We'll visualize the middle slice of each channel
#     fig, axes = plt.subplots(2, 7, figsize=(20, 10))
#     image_names = ["flair","t1","t1ce","t2"]
#     for channel in range(4):
#         # Original image
#         axes[0,channel].imshow(images[0, channel, :, :, 64].cpu().numpy(), cmap='gray')
#         axes[0,channel].set_title(image_names[channel])
#         axes[0,channel].axis('off')
        
    
#     for i in range(4,7):
#         axes[0,i].imshow(cam_images[-4+i][0, 0, :, :, 64].cpu().numpy())
#         axes[0,i].set_title(cam_image_names[-4+i])
#         axes[0,i].axis('off')
        
#     for channel in range(4):
#         # Original image
#         axes[1,channel].imshow(images[0, channel, :, :, 64].cpu().numpy(), cmap='gray')
#         axes[1,channel].set_title(image_names[channel])
#         axes[1,channel].axis('off')
        
    
#     for i in range(7,10):
#         axes[1,i-3].imshow(cam_images[-4+i][0, 0, :, :, 64].cpu().numpy())
#         axes[1,i-3].set_title(cam_image_names[-4+i])
#         axes[1,i-3].axis('off')
        
        
    
#     plt.tight_layout()
#     plt.show()

In [ ]:
# from monai.visualize import GradCAM
# import matplotlib.pyplot as plt

# # Assuming 'saved_model' is your loaded model
# gcam = GradCAM(nn_module=saved_model.net, target_layers="out")

# for batch_idx,batch in enumerate(resized_val_loader):
#     images = batch["image"].to('cuda')
#     images = torch.squeeze(images,dim=0)
    
    
    
#     # Generate GradCAM
#     cam0 = gcam(x=images, class_idx=0)
#     cam1 = gcam(x=images, class_idx=1)
#     cam2 = gcam(x=images, class_idx=2)
    
    
#     print(cam.shape)
    
#     # Visualize the GradCAM results
#     # We'll visualize the middle slice of each channel
#     fig, axes = plt.subplots(1, 7, figsize=(20, 10))
#     image_names = ["flair","t1","t1ce","t2"]
#     for channel in range(4):
#         # Original image
#         axes[channel].imshow(images[0, channel, :, :, 64].cpu().numpy(), cmap='gray')
#         axes[channel].set_title(image_names[channel])
#         axes[channel].axis('off')
        
#         # GradCAM
#     axes[4].imshow(cam0[0, 0, :, :, 64].cpu().numpy())
#     axes[4].set_title(f'GradCAM Class 0')
#     axes[4].axis('off')
    
#     axes[5].imshow(cam1[0, 0, :, :, 64].cpu().numpy())
#     axes[5].set_title(f'GradCAM Class 1')
#     axes[5].axis('off')
    
#     axes[6].imshow(cam2[0, 0, :, :, 64].cpu().numpy())
#     axes[6].set_title(f'GradCAM Class 2')
#     axes[6].axis('off')
    
#     plt.tight_layout()
#     plt.show()